# Pandas

Материалы:
* Макрушин С.В. "Лекция 2: Библиотека Pandas"
* https://pandas.pydata.org/docs/user_guide/index.html#
* https://pandas.pydata.org/docs/reference/index.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Загрузите данные из файла `sp500hst.txt` и обозначьте столбцы в соответствии с содержимым: `"date", "ticker", "open", "high", "low", "close", "volume"`.

2. Рассчитайте среднее значение показателей для каждого из столбцов c номерами 3-6.

3. Добавьте столбец, содержащий только число месяца, к которому относится дата.

4. Рассчитайте суммарный объем торгов для для одинаковых значений тикеров.

5. Загрузите данные из файла sp500hst.txt и обозначьте столбцы в соответствии с содержимым: "date", "ticker", "open", "high", "low", "close", "volume". Добавьте столбец с расшифровкой названия тикера, используя данные из файла `sp_data2.csv` . В случае нехватки данных об именах тикеров корректно обработать их.

## Лабораторная работа №2

### Базовые операции с `DataFrame`

In [ ]:
import pandas as pd
import numpy as np

1.1 В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца с индексами в таблице `reviews` (безымянный столбец).

In [ ]:
recipes = pd.read_csv("data/recipes_sample.csv")
reviews = pd.read_csv("data/reviews_sample.csv", index_col=0)

1.2 Для каждой из таблиц выведите основные параметры:
* количество точек данных (строк);
* количество столбцов;
* тип данных каждого столбца.

In [ ]:
# Count of rows
print(f"Количество строк с рецептами: {recipes.shape[0]}")
print(f"Количество строк с отзывами: {reviews.count()[0]}")

# Count of cols
print(f"Количество столбцов с рецептами: {recipes.shape[1]}")
print(f"Количество столбцов с отзывами: {len(reviews.iloc[0])}")

# Type of cols
new_line = '\n'
print(f"Типы данных для столбцов с рецептами: {new_line.join(ind + ':' + str(recipes.dtypes[ind]) for ind in recipes.dtypes.index)}")
print(f"Типы данных для столбцов с отзывами: {new_line.join(ind + ':' + str(reviews.dtypes[ind]) for ind in reviews.dtypes.index)}")



1.3 Исследуйте, в каких столбцах таблиц содержатся пропуски. Посчитайте долю строк, содержащих пропуски, в отношении к общему 
количеству строк.

In [ ]:
print(f"Пропуски в таблице с рецептами содержатся в след. столбцах:")
for idx, val in enumerate(recipes.isna().sum()):
    if val > 0:
        print(recipes.columns[idx])
print(f'Доля строк с пропусками: {(recipes.shape[0] - recipes.dropna().shape[0]) / recipes.shape[0]:.2f}\n')

print(f"Пропуски в таблице с отзывами содержатся в след. столбцах:")
print('\n'.join(i for i in reviews.columns[reviews.isna().sum() > 0]))

print(f'Доля строк с пропусками: {reviews.isna().any(axis=1).sum() / reviews.shape[0]:.5f}\n')

1.4 Рассчитайте среднее значение для каждого из числовых столбцов (где это имеет смысл).

In [ ]:
for i in recipes.dtypes.index:
    if ('int' in str(recipes.dtypes[i]) or 'float' in str(recipes.dtypes[i])) and 'id' not in i:
        print(f"Среднее значение для столбца {i} - {recipes[i].mean(): .2f}")

1.5 Создайте серию из 10 случайных названий рецептов.

In [ ]:
print("Названия 10 случайных рецептов:")
random_recipes = recipes['name'].sample(n=10, replace=False)
print("\n".join(f"{i:<}" for i in random_recipes))

1.6 Измените индекс в таблице `reviews`, пронумеровав строки, начиная с нуля.

In [ ]:
reviews.reset_index(inplace=True, drop=True)
reviews

1.7 Выведите информацию о рецептах, время выполнения которых не больше 20 минут и кол-во ингредиентов в которых не больше 5.

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None):
    display(recipes[(recipes['minutes'] <= 20) & (recipes['n_ingredients'] <= 5)])

### Работа с датами в `pandas`

2.1 Преобразуйте столбец `submitted` из таблицы `recipes` в формат времени. Модифицируйте решение задачи 1.1 так, чтобы считать столбец сразу в нужном формате.

In [ ]:
# Convert column
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
print(f"Результат конвертирования {recipes.dtypes['submitted'].name}")

In [ ]:
# Reading file with datetime column
recipes_modified = pd.read_csv("data/recipes_sample.csv", parse_dates=['submitted'])
print(f"Результат создания DataFrame из файла с обработкой даты: {recipes_modified.dtypes['submitted'].name}")

2.2 Выведите информацию о рецептах, добавленных в датасет не позже 2010 года.

In [ ]:
# Date of start 2010 year
border_date = np.datetime64("2010-01-01")
display(recipes[recipes['submitted'] <= border_date])

### Работа со строковыми данными в `pandas`

3.1  Добавьте в таблицу `recipes` столбец `description_length`, в котором хранится длина описания рецепта из столбца `description`.

In [ ]:
recipes['description_length'] = recipes['description'].str.len()

recipes

3.2 Измените название каждого рецепта в таблице `recipes` таким образом, чтобы каждое слово в названии начиналось с прописной буквы.

In [ ]:
recipes.name = recipes['name'].str.title()

recipes

3.3 Добавьте в таблицу `recipes` столбец `name_word_count`, в котором хранится количество слов из названии рецепта (считайте, что слова в названии разделяются только пробелами). Обратите внимание, что между словами может располагаться несколько пробелов подряд.

In [ ]:
recipes['name_word_count'] = recipes['name'].str.split().map(len)

recipes

### Группировки таблиц `pd.DataFrame`

4.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [ ]:
for ind, value in recipes.groupby('contributor_id')['id'].count().items():
    print(f"ID участника: {ind}, кол-во рецептов: {value}")

In [ ]:
f"ID участника, добавившего больше всех рецептов: {recipes.groupby('contributor_id').count()['name'].idxmax()}"

4.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [ ]:
for (recipe_id, avg) in reviews.groupby('recipe_id')['rating'].mean().items():
    print(f"Средний рейтинг у рецепта {recipe_id} = {avg: .2f}")

In [ ]:
f"Кол-во рецептов без отзыва {len(recipes) - len(reviews.groupby('recipe_id').count())}"

4.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [ ]:
for (year, cnt) in recipes.groupby(recipes.submitted.dt.year).count()['id'].items():
    print(f"Год: {year}, кол-во рецептов: {cnt}")

### Объединение таблиц `pd.DataFrame`

5.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
df = pd.merge(recipes[['id', 'name']], reviews[['recipe_id', 'user_id', 'rating']], left_on=['id'], right_on=['recipe_id'], how='right')
df.drop('recipe_id', axis=1, inplace=True)

In [ ]:
# Как найти рецепт без отзыва
df2 = pd.merge(recipes[['id', 'name']], reviews[['recipe_id', 'user_id', 'rating']], left_on=['id'], right_on=['recipe_id'], how='left', indicator=True)
df2.query("_merge == 'left_only'")

In [ ]:
# Отзыва на рецепт с таким ID нет.
df[df['id'] == 223349]

In [ ]:
# Отзыв на такой рецепт имеется (даже много).
df[df['id'] == 57993]

5.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
df3 = pd.merge(recipes[['id', 'name']], reviews.groupby('recipe_id').count()['review'], left_on=['id'], right_on=['recipe_id'], how='left')
df3.columns = ['recipe_id', 'name', 'review_count']
df3.fillna(0, axis=1, inplace=True)

df3[df3['recipe_id'] == 223349]

5.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [ ]:
df4 = pd.merge(recipes[['id', 'submitted']], reviews[['recipe_id', 'rating']], left_on=['id'], right_on=['recipe_id'], how='right')
df4['submitted'] = pd.to_datetime(df4['submitted'])

avg_rate_by_year = df4.groupby(df4.submitted.dt.year)['rating'].mean()

f"Наименьший средний рейтинг ({avg_rate_by_year.min():.2f}) был зафиксирован в {avg_rate_by_year.idxmin()}"

### Сохранение таблиц `pd.DataFrame`

6.1 Отсортируйте таблицу в порядке убывания величины столбца `name_word_count` и сохраните результаты выполнения заданий 3.1-3.3 в csv файл. 

In [ ]:
recipes.sort_values(by='name_word_count', ascending=False, inplace=True)

recipes.to_csv("data/recipes_result.csv")

6.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 5.1 и 5.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 5.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 5.2.

In [ ]:
import openpyxl
with pd.ExcelWriter("data/recipes_result.xlsx") as writer:
    df.to_excel(writer, sheet_name="Рецепты с оценками")  
    df3.to_excel(writer, sheet_name="Количество отзывов по рецептам")  

#### [версия 2]
* Уточнены формулировки задач 1.1, 3.3, 4.2, 5.1, 5.2, 5.3